In [56]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import xarray as xr
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn.image import clean_img
from nilearn.interfaces.fmriprep import load_confounds_strategy
from tqdm import tqdm
from nilearn.datasets import fetch_atlas_schaefer_2018


## Preparing target data

In [5]:
participants = pd.read_csv("/data/parietal/store2/data/camcan1366/cc700/mri/pipeline/release004/BIDS_20190411/epi_rest/participants.tsv", sep = '\t')

In [22]:
participants

,participant_id,age,hand,gender_text,gender_code,tiv_cubicmm
0,sub-CC110033,24,90.0,MALE,1,1411686
1,sub-CC110037,18,89.0,MALE,1,1391950
2,sub-CC110045,24,100.0,FEMALE,2,1384737
3,sub-CC110056,22,100.0,FEMALE,2,1284916
4,sub-CC110069,28,-58.0,FEMALE,2,1417211
...,...,...,...,...,...,...
647,sub-CC722542,79,100.0,MALE,1,1594938
648,sub-CC722651,79,100.0,FEMALE,2,1388869
649,sub-CC722891,84,89.0,FEMALE,2,1416929
650,sub-CC723197,80,100.0,FEMALE,2,1444360


In [7]:
participants_2 = pd.read_csv("/home/mind/alebris/projects/pavi_project/data/camcan/scores_camcan.csv")

In [23]:
participants_2

,Unnamed: 0,Subject,BentonFaces_total,CardioMeasures_pulse_mean,CardioMeasures_bp_sys_mean,CardioMeasures_bp_dia_mean,Cattell_total,EkmanEmHex_pca1,EkmanEmHex_pca1_expv,FamousFaces_details,...,Proverbs,RTchoice,RTsimple,Synsem_prop_error,Synsem_RT,TOT,VSTMcolour_K_mean,VSTMcolour_K_precision,VSTMcolour_K_doubt,VSTMcolour_MSE
0,0,CC210148,23.0,NaN,NaN,NaN,41.0,-396.223762,0.657152,0.950000,...,2.0,NaN,NaN,0.147870,1757.9,0.500000,2.166470,0.429817,14.905354,2133.128000
1,1,CC110033,20.0,65.0,111.5,66.0,42.0,375.446574,0.657152,0.964286,...,5.0,0.474851,0.302018,0.047619,1712.0,0.368421,2.445633,0.551437,40.378580,1116.788000
2,2,CC110037,21.0,NaN,NaN,NaN,30.0,1609.716091,0.657152,0.983333,...,6.0,0.500034,0.348958,0.384618,1353.7,0.128205,1.874599,0.452397,56.146430,3150.940909
3,3,CC110045,23.0,61.5,90.5,63.5,41.0,-658.667678,0.657152,0.933333,...,5.0,0.459115,0.310046,0.219048,1808.4,0.333333,2.535944,0.543763,24.744630,696.120000
4,4,CC110056,25.0,NaN,NaN,NaN,24.0,-355.959812,0.657152,0.983333,...,1.0,0.682312,0.402969,0.279167,1408.6,0.227273,2.000551,0.456191,32.694635,2690.847273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,548,CC722542,23.0,51.0,149.5,72.0,23.0,-37.905938,0.657152,0.933333,...,6.0,0.659964,0.417380,0.095238,1592.7,0.575758,1.834328,0.327941,23.007149,3627.339818
549,549,CC722651,24.0,60.5,122.5,63.5,28.0,-1539.327925,0.657152,0.940000,...,6.0,0.551149,0.347513,0.047619,1385.5,0.500000,1.960161,0.494379,21.646447,2729.643000
550,550,CC722891,18.0,76.0,136.5,73.0,21.0,1384.253987,0.657152,0.900000,...,4.0,0.681643,0.473639,0.231610,2174.5,0.714286,1.570112,0.399219,1.394645,3782.428909
551,551,CC723197,22.0,64.0,186.0,94.0,27.0,-25.661712,0.657152,0.608696,...,6.0,0.755996,0.420891,0.196577,2118.4,0.937500,1.881676,0.405702,17.514300,3512.471727


In [24]:
participants['Subject'] = participants['participant_id'].str.replace('sub-', '', regex=False)

# Step 2: Merge DataFrames based on the matching Subject IDs
merged = pd.merge(participants_2, participants[['Subject', 'age']], on='Subject', how='left')


In [25]:
merged

,Unnamed: 0,Subject,BentonFaces_total,CardioMeasures_pulse_mean,CardioMeasures_bp_sys_mean,CardioMeasures_bp_dia_mean,Cattell_total,EkmanEmHex_pca1,EkmanEmHex_pca1_expv,FamousFaces_details,...,RTchoice,RTsimple,Synsem_prop_error,Synsem_RT,TOT,VSTMcolour_K_mean,VSTMcolour_K_precision,VSTMcolour_K_doubt,VSTMcolour_MSE,age
0,0,CC210148,23.0,NaN,NaN,NaN,41.0,-396.223762,0.657152,0.950000,...,NaN,NaN,0.147870,1757.9,0.500000,2.166470,0.429817,14.905354,2133.128000,30
1,1,CC110033,20.0,65.0,111.5,66.0,42.0,375.446574,0.657152,0.964286,...,0.474851,0.302018,0.047619,1712.0,0.368421,2.445633,0.551437,40.378580,1116.788000,24
2,2,CC110037,21.0,NaN,NaN,NaN,30.0,1609.716091,0.657152,0.983333,...,0.500034,0.348958,0.384618,1353.7,0.128205,1.874599,0.452397,56.146430,3150.940909,18
3,3,CC110045,23.0,61.5,90.5,63.5,41.0,-658.667678,0.657152,0.933333,...,0.459115,0.310046,0.219048,1808.4,0.333333,2.535944,0.543763,24.744630,696.120000,24
4,4,CC110056,25.0,NaN,NaN,NaN,24.0,-355.959812,0.657152,0.983333,...,0.682312,0.402969,0.279167,1408.6,0.227273,2.000551,0.456191,32.694635,2690.847273,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,548,CC722542,23.0,51.0,149.5,72.0,23.0,-37.905938,0.657152,0.933333,...,0.659964,0.417380,0.095238,1592.7,0.575758,1.834328,0.327941,23.007149,3627.339818,79
549,549,CC722651,24.0,60.5,122.5,63.5,28.0,-1539.327925,0.657152,0.940000,...,0.551149,0.347513,0.047619,1385.5,0.500000,1.960161,0.494379,21.646447,2729.643000,79
550,550,CC722891,18.0,76.0,136.5,73.0,21.0,1384.253987,0.657152,0.900000,...,0.681643,0.473639,0.231610,2174.5,0.714286,1.570112,0.399219,1.394645,3782.428909,84
551,551,CC723197,22.0,64.0,186.0,94.0,27.0,-25.661712,0.657152,0.608696,...,0.755996,0.420891,0.196577,2118.4,0.937500,1.881676,0.405702,17.514300,3512.471727,80


In [26]:
targets = merged.dropna()


In [27]:
targets

,Unnamed: 0,Subject,BentonFaces_total,CardioMeasures_pulse_mean,CardioMeasures_bp_sys_mean,CardioMeasures_bp_dia_mean,Cattell_total,EkmanEmHex_pca1,EkmanEmHex_pca1_expv,FamousFaces_details,...,RTchoice,RTsimple,Synsem_prop_error,Synsem_RT,TOT,VSTMcolour_K_mean,VSTMcolour_K_precision,VSTMcolour_K_doubt,VSTMcolour_MSE,age
1,1,CC110033,20.0,65.0,111.5,66.0,42.0,375.446574,0.657152,0.964286,...,0.474851,0.302018,0.047619,1712.0,0.368421,2.445633,0.551437,40.378580,1116.788000,24
3,3,CC110045,23.0,61.5,90.5,63.5,41.0,-658.667678,0.657152,0.933333,...,0.459115,0.310046,0.219048,1808.4,0.333333,2.535944,0.543763,24.744630,696.120000,24
5,5,CC110069,23.0,62.0,94.5,61.5,41.0,-770.282040,0.657152,0.666667,...,0.358177,0.298550,0.190476,903.5,0.000000,2.405834,0.595672,10.935720,1131.691091,28
7,7,CC110411,22.0,61.5,92.5,54.0,43.0,-671.348832,0.657152,0.708333,...,0.375033,0.304480,0.023810,1160.2,0.666667,2.230150,0.395571,39.464270,2211.743727,25
9,9,CC112141,24.0,66.0,109.5,64.5,43.0,137.114456,0.657152,1.000000,...,0.476920,0.305420,0.252381,1450.4,0.400000,2.173543,0.519179,44.430360,1856.463273,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,546,CC722421,20.0,69.5,162.5,88.5,28.0,1330.176019,0.657152,0.857143,...,0.601125,0.370109,0.214286,1510.5,0.857143,1.837692,0.497337,3.466072,2953.141000,79
548,548,CC722542,23.0,51.0,149.5,72.0,23.0,-37.905938,0.657152,0.933333,...,0.659964,0.417380,0.095238,1592.7,0.575758,1.834328,0.327941,23.007149,3627.339818,79
549,549,CC722651,24.0,60.5,122.5,63.5,28.0,-1539.327925,0.657152,0.940000,...,0.551149,0.347513,0.047619,1385.5,0.500000,1.960161,0.494379,21.646447,2729.643000,79
550,550,CC722891,18.0,76.0,136.5,73.0,21.0,1384.253987,0.657152,0.900000,...,0.681643,0.473639,0.231610,2174.5,0.714286,1.570112,0.399219,1.394645,3782.428909,84


## Preprocessing nifti files, and turning them into connectivity matrices

In [28]:
n_parcels = 400  
bold_tr = 2.0    
#nifti_folder = 'nifti_camcan'  


In [29]:
# Fetch atlas
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=n_parcels, yeo_networks=7, resolution_mm=2, base_url=None, resume=True, verbose=1)


 ...done. (0 seconds, 0 min)


In [30]:
#Initialize the masker
masker = NiftiLabelsMasker(labels_img=atlas.maps, standardize="zscore_sample",
                            standardize_confounds="zscore_sample", memory='nilearn_cache',
                            verbose=5, smoothing_fwhm=2, low_pass=0.08, high_pass=0.009,
                            t_r=bold_tr)

In [40]:
nifti_base_path = "/data/parietal/store2/derivatives/camcan-fmriprep"
output_path = "/data/parietal/store2/work/mrenaudi/contrastive-reg-3/connectivity_matrices"


In [41]:
for subject_id in tqdm(targets['Subject'], desc="Processing Subjects"):
    # Construct the NIfTI file path
    nifti_file = os.path.join(nifti_base_path, f"sub-{subject_id}", 'func', f"sub-{subject_id}_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")
    
    
    try:
        # Load and preprocess the NIfTI file
        confounds_sample, sample_mask = load_confounds_strategy(
            nifti_file,
            strategy=["high_pass", "motion", "wm_csf", "scrub"],
            motion="full",
            wm_csf="basic",
            scrub=5,
            fd_threshold=0.2
        )
        
        confounds_sample = confounds_sample.drop(columns=['cosine00', 'cosine01', 'cosine02'])  # Remove specified columns

        # Extract parcel time series
        parc_ts = masker.fit_transform(nifti_file, confounds=confounds_sample, sample_mask=sample_mask)
        
        # Calculate the correlation matrix
        correlation_measure = ConnectivityMeasure(kind="correlation", standardize=True)
        conn_matrix = correlation_measure.fit_transform([parc_ts])[0]

        # Save the connectivity matrix as a .npy file
        np.save(os.path.join(output_path, f"connectivity_matrix_sub-{subject_id}.npy"), conn_matrix)

    except Exception as e:
        print(f"Error processing subject {subject_id}: {e}")

Processing Subjects:   0%|          | 0/352 [00:00<?, ?it/s]/home/mind/mrenaudi/.local/miniconda3/lib/python3.12/site-packages/nilearn/interfaces/fmriprep/load_confounds_strategy.py:222: UserWarning: The following parameters are not needed for the selected strategy 'simple': ['strategy', 'scrub', 'fd_threshold']; parameters accepted: ['motion', 'wm_csf', 'global_signal', 'demean']
  warnings.warn(


[NiftiLabelsMasker.wrapped] loading data from /home/mind/mrenaudi/nilearn_data/schaefer_2018/Schaefer2018_400Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
[Memory]1019.9s, 17.0min: Loading _filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min


Processing Subjects:   0%|          | 1/352 [00:03<21:18,  3.64s/it]/home/mind/mrenaudi/.local/miniconda3/lib/python3.12/site-packages/nilearn/interfaces/fmriprep/load_confounds_strategy.py:222: UserWarning: The following parameters are not needed for the selected strategy 'simple': ['strategy', 'scrub', 'fd_threshold']; parameters accepted: ['motion', 'wm_csf', 'global_signal', 'demean']
  warnings.warn(


[NiftiLabelsMasker.wrapped] loading data from /home/mind/mrenaudi/nilearn_data/schaefer_2018/Schaefer2018_400Parcels_7Networks_order_FSLMNI152_2mm.nii.gz


Processing Subjects:   1%|          | 2/352 [00:07<21:12,  3.63s/it]

...

/home/mind/mrenaudi/.local/miniconda3/lib/python3.12/site-packages/nilearn/signal.py:872: FutureWarning: By default the cubic spline interpolator extrapolates the out-of-bounds censored volumes in the data run. This can lead to undesired filtered signal results. Starting in version 0.13, the default strategy will be not to extrapolate but to discard those volumes at filtering.
  warnings.warn(
Processing Subjects: 100%|█████████▉| 351/352 [56:23<00:10, 10.12s/it]

[NiftiLabelsMasker.transform_single_imgs] Cleaning extracted signals
_______________________________________________filter_and_extract - 6.4s, 0.1min


/home/mind/mrenaudi/.local/miniconda3/lib/python3.12/site-packages/nilearn/interfaces/fmriprep/load_confounds_strategy.py:222: UserWarning: The following parameters are not needed for the selected strategy 'simple': ['strategy', 'scrub', 'fd_threshold']; parameters accepted: ['motion', 'wm_csf', 'global_signal', 'demean']
  warnings.warn(


[NiftiLabelsMasker.wrapped] loading data from /home/mind/mrenaudi/nilearn_data/schaefer_2018/Schaefer2018_400Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
________________________________________________________________________________
[Memory] Calling nilearn.maskers.base_masker._filter_and_extract...
_filter_and_extract('/data/parietal/store2/derivatives/camcan-fmriprep/sub-CC723197/func/sub-CC723197_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', 
{ 'background_label': 0,
  'clean_kwargs': {},
  'detrend': False,
  'dtype': None,
  'high_pass': 0.009,
  'high_variance_confounds': False,
  'keep_masked_labels': True,
  'labels': None,
  'labels_img': '/home/mind/mrenaudi/nilearn_data/schaefer_2018/Schaefer2018_400Parcels_7Networks_order_FSLMNI152_2mm.nii.gz',
  'low_pass': 0.08,
  'mask_img': None,
  'reports': True,
  'smoothing_fwhm': 2,
  'standardize': 'zscore_sample',
  'standardize_confounds': 'zscore_sample',
  'strategy': 'mean',
  't_r': 2.0,
  'target_affine':

/home/mind/mrenaudi/.local/miniconda3/lib/python3.12/site-packages/nilearn/signal.py:872: FutureWarning: By default the cubic spline interpolator extrapolates the out-of-bounds censored volumes in the data run. This can lead to undesired filtered signal results. Starting in version 0.13, the default strategy will be not to extrapolate but to discard those volumes at filtering.
  warnings.warn(
Processing Subjects: 100%|██████████| 352/352 [56:34<00:00,  9.64s/it]


## Creating xarray dataset

In [42]:
connectivity_matrix_dir = 'connectivity_matrices'


In [43]:
connectivity_matrices = []
subject_ids = []

In [44]:
for subject_id in targets['Subject']:
    # Construct the file path for each subject's connectivity matrix
    conn_matrix_path = os.path.join(connectivity_matrix_dir, f"connectivity_matrix_sub-{subject_id}.npy")
    
    # Check if the file exists before loading
    if os.path.exists(conn_matrix_path):
        # Load the connectivity matrix
        conn_matrix = np.load(conn_matrix_path)
        
        # Append the matrix and the subject ID
        connectivity_matrices.append(conn_matrix)
        subject_ids.append(subject_id)
    else:
        print(f"Warning: Connectivity matrix for subject {subject_id} not found at {conn_matrix_path}")


In [63]:
matrices = np.array(connectivity_matrices)


In [64]:
matrices.shape

(352, 400, 400)

In [46]:
# Print the column names and data types for the targets DataFrame
print("Column names and data types in targets DataFrame:")
for column in targets.columns:
    print(f"{column}: {targets[column].dtype}")


Column names and data types in targets DataFrame:
Unnamed: 0: int64
Subject: object
BentonFaces_total: float64
CardioMeasures_pulse_mean: float64
CardioMeasures_bp_sys_mean: float64
CardioMeasures_bp_dia_mean: float64
Cattell_total: float64
EkmanEmHex_pca1: float64
EkmanEmHex_pca1_expv: float64
FamousFaces_details: float64
Hotel_time: float64
PicturePriming_baseline_acc: float64
PicturePriming_baseline_rt: float64
PicturePriming_priming_prime: float64
PicturePriming_priming_target: float64
Proverbs: float64
RTchoice: float64
RTsimple: float64
Synsem_prop_error: float64
Synsem_RT: float64
TOT: float64
VSTMcolour_K_mean: float64
VSTMcolour_K_precision: float64
VSTMcolour_K_doubt: float64
VSTMcolour_MSE: float64
age: int64


In [55]:
participant_id = targets["Subject"].astype("str").values

benton_faces = targets["BentonFaces_total"].astype("float").values

cardio_measure_pulse_mean = targets["CardioMeasures_pulse_mean"].astype("float").values
cardio_measure_bp_sys_mean = targets["CardioMeasures_bp_sys_mean"].astype("float").values
cardio_measure_bp_dia_mean = targets["CardioMeasures_bp_dia_mean"].astype("float").values

cattell = targets["Cattell_total"].astype("float").values

ekman_em_hex_pca1 = targets["EkmanEmHex_pca1"].astype("float").values
ekman_em_hex_pca1_expv = targets["EkmanEmHex_pca1_expv"].astype("float").values

famous_faces = targets["FamousFaces_details"].astype("float").values

hotel_time = targets["Hotel_time"].astype("float").values

picture_priming_baseline_acc = targets["PicturePriming_baseline_acc"].astype("float").values
picture_priming_baseline_rt = targets["PicturePriming_baseline_rt"].astype("float").values
picture_priming_priming_prime = targets["PicturePriming_priming_prime"].astype("float").values
picture_priming_priming_target = targets["PicturePriming_priming_target"].astype("float").values

proverbs = targets["Proverbs"].astype("float").values

rt_choice = targets["RTchoice"].astype("float").values
rt_simple = targets["RTsimple"].astype("float").values

syn_sem_prop_error = targets["Synsem_prop_error"].astype("float").values
syn_sem_rt = targets["Synsem_RT"].astype("float").values

tip_of_tongue = targets["TOT"].astype("float").values

VSTM_colour_K_mean = targets["VSTMcolour_K_mean"].astype("float").values
VSTM_colour_K_precision = targets["VSTMcolour_K_precision"].astype("float").values
VSTM_colour_K_doubt = targets["VSTMcolour_K_doubt"].astype("float").values
VSTM_colour_MSE = targets["VSTMcolour_MSE"].astype("float").values

age = targets["age"].astype("float").values


In [57]:
atlas_labels = pd.Series(fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=7)['labels']).astype("str").values

In [65]:
dataset = xr.DataArray(
    data=matrices,
    dims=["subject", "parcel_1", "parcel_2"],
    coords=dict(
        participant_id=(["subject"], participant_id),
        age=(["subject"], age),
        benton_faces =(["subject"], benton_faces),
        cardio_measure_pulse_mean=(["subject"], cardio_measure_pulse_mean),
        cardio_measure_bp_sys_mean=(["subject"], cardio_measure_bp_sys_mean),
        cardio_measure_bp_dia_mean=(["subject"], cardio_measure_bp_dia_mean),
        cattell=(["subject"], cattell),
        ekman_em_hex_pca1=(["subject"], ekman_em_hex_pca1),
        ekman_em_hex_pca1_expv=(["subject"], ekman_em_hex_pca1_expv),
        famous_faces=(["subject"], famous_faces),
        hotel_time=(["subject"], hotel_time),
        picture_priming_baseline_acc=(["subject"], picture_priming_baseline_acc),
        picture_priming_baseline_rt=(["subject"], picture_priming_baseline_rt),
        picture_priming_priming_prime=(["subject"], picture_priming_priming_prime),
        picture_priming_priming_target=(["subject"], picture_priming_priming_target),
        proverbs=(["subject"], proverbs),
        rt_choice=(["subject"], rt_choice),
        rt_simple=(["subject"], rt_simple),
        syn_sem_prop_error=(["subject"], syn_sem_prop_error),
        syn_sem_rt=(["subject"], syn_sem_rt),
        tip_of_tongue=(["subject"], tip_of_tongue),
        VSTM_colour_K_mean=(["subject"], VSTM_colour_K_mean),
        VSTM_colour_K_precision=(["subject"], VSTM_colour_K_precision),
        VSTM_colour_K_doubt=(["subject"], VSTM_colour_K_doubt),
        VSTM_colour_MSE=(["subject"], VSTM_colour_MSE),
        parc1=(["parcel_1"], atlas_labels),
        parc2=(["parcel_2"], atlas_labels),
    ),
    attrs=dict(
        description="Matrix dataset; Parcellation: Schaefer 400"
    ),
)

In [70]:
dataset = xr.Dataset(
    {
        'matrices': (["subject", "parcel_1", "parcel_2"], matrices),
    },
    coords=dict(
        participant_id=(["subject"], participant_id),
        age=(["subject"], age),
        benton_faces =(["subject"], benton_faces),
        cardio_measure_pulse_mean=(["subject"], cardio_measure_pulse_mean),
        cardio_measure_bp_sys_mean=(["subject"], cardio_measure_bp_sys_mean),
        cardio_measure_bp_dia_mean=(["subject"], cardio_measure_bp_dia_mean),
        cattell=(["subject"], cattell),
        ekman_em_hex_pca1=(["subject"], ekman_em_hex_pca1),
        ekman_em_hex_pca1_expv=(["subject"], ekman_em_hex_pca1_expv),
        famous_faces=(["subject"], famous_faces),
        hotel_time=(["subject"], hotel_time),
        picture_priming_baseline_acc=(["subject"], picture_priming_baseline_acc),
        picture_priming_baseline_rt=(["subject"], picture_priming_baseline_rt),
        picture_priming_priming_prime=(["subject"], picture_priming_priming_prime),
        picture_priming_priming_target=(["subject"], picture_priming_priming_target),
        proverbs=(["subject"], proverbs),
        rt_choice=(["subject"], rt_choice),
        rt_simple=(["subject"], rt_simple),
        syn_sem_prop_error=(["subject"], syn_sem_prop_error),
        syn_sem_rt=(["subject"], syn_sem_rt),
        tip_of_tongue=(["subject"], tip_of_tongue),
        VSTM_colour_K_mean=(["subject"], VSTM_colour_K_mean),
        VSTM_colour_K_precision=(["subject"], VSTM_colour_K_precision),
        VSTM_colour_K_doubt=(["subject"], VSTM_colour_K_doubt),
        VSTM_colour_MSE=(["subject"], VSTM_colour_MSE),
        parc1=(["parcel_1"], atlas_labels),
        parc2=(["parcel_2"], atlas_labels),
    ),
    attrs=dict(
        description="Matrix dataset; Parcellation: Schaefer 400"
    ),
)

In [71]:
dataset.to_netcdf("dataset_400parcels_2.nc")

In [72]:
dataset = xr.open_dataset("dataset_400parcels_2.nc")

In [73]:
dataset

<xarray.Dataset> Size: 225MB
Dimensions:                         (parcel_1: 400, parcel_2: 400, subject: 352)
Coordinates: (12/27)
    parc1                           (parcel_1) object 3kB ...
    parc2                           (parcel_2) object 3kB ...
    participant_id                  (subject) object 3kB ...
    age                             (subject) float64 3kB ...
    benton_faces                    (subject) float64 3kB ...
    cardio_measure_pulse_mean       (subject) float64 3kB ...
    ...                              ...
    syn_sem_rt                      (subject) float64 3kB ...
    tip_of_tongue                   (subject) float64 3kB ...
    VSTM_colour_K_mean              (subject) float64 3kB ...
    VSTM_colour_K_precision         (subject) float64 3kB ...
    VSTM_colour_K_doubt             (subject) float64 3kB ...
    VSTM_colour_MSE                 (subject) float64 3kB ...
Dimensions without coordinates: parcel_1, parcel_2, subject
Data variables:
    matrices                        (subject, parcel_1, parcel_2) float32 225MB ...
Attributes:
    description:  Matrix dataset; Parcellation: Schaefer 400

In [74]:
dataset.matrices

<xarray.DataArray 'matrices' (subject: 352, parcel_1: 400, parcel_2: 400)> Size: 225MB
[56320000 values with dtype=float32]
Coordinates: (12/27)
    parc1                           (parcel_1) object 3kB ...
    parc2                           (parcel_2) object 3kB ...
    participant_id                  (subject) object 3kB ...
    age                             (subject) float64 3kB ...
    benton_faces                    (subject) float64 3kB ...
    cardio_measure_pulse_mean       (subject) float64 3kB ...
    ...                              ...
    syn_sem_rt                      (subject) float64 3kB ...
    tip_of_tongue                   (subject) float64 3kB ...
    VSTM_colour_K_mean              (subject) float64 3kB ...
    VSTM_colour_K_precision         (subject) float64 3kB ...
    VSTM_colour_K_doubt             (subject) float64 3kB ...
    VSTM_colour_MSE                 (subject) float64 3kB ...
Dimensions without coordinates: subject, parcel_1, parcel_2